In [19]:
import os
import pandas as pd
from glob import glob
from collections import defaultdict
from chardet import detect

0. 인코딩 (파일 옮긴 후에 거기 디렉토리 잡아서 하기)

In [189]:
# 경로 설정
input_dir = "C:/Users/user/Downloads"
output_dir = "./"
os.makedirs(output_dir, exist_ok=True)

# 처리할 기준년월 범위
target_months = [f"인구_2025{m:02d}.csv" for m in range(1, 5)]
# target_months = [f"인구_202503.csv"]

# 인코딩 확인 및 UTF-8로 변환
def convert_to_utf8(file_path):
    with open(file_path, 'rb') as f:
        raw = f.read()
        enc = detect(raw)['encoding']
        print(f'전 : {file}: {enc}')
    if enc.lower() != 'utf-8':
        new_path = file_path
        with open(file_path, 'r', encoding=enc, errors='ignore') as f:
            text = f.read()
        with open(new_path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f'후 : {file}: {detect(f.read())['encoding']}')
        

for file in target_months:
    full_path = os.path.join(input_dir, file)
    convert_to_utf8(full_path)

전 : 인구_202501.csv: utf-8
전 : 인구_202502.csv: utf-8
전 : 인구_202503.csv: utf-8
전 : 인구_202504.csv: utf-8


1. 월별레코드수 확인 + 합치기(df_all)

In [190]:
dfs = []
for file in target_months:
    df = pd.read_csv(os.path.join(input_dir, file), encoding='utf-8')
    print(f"처리전 {file} 레코드수 : {len(df)}")
    df['파일명'] = file
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

처리전 인구_202501.csv 레코드수 : 1000000
처리전 인구_202502.csv 레코드수 : 1000000
처리전 인구_202503.csv 레코드수 : 1000000
처리전 인구_202504.csv 레코드수 : 1000000


2. 가명처리 함수 정의

In [191]:
def process_age(df):
    def encode_age(age):
        if pd.isna(age):
            return None
        try:
            age = int(age)
            if age < 10:
                return 1
            elif age >= 70:
                return 14
            else:
                return (age - 10) // 5 + 2
        except ValueError:
            return None
    df['연령대'] = df['연령'].apply(encode_age)
    df= df.drop(columns=['연령'])
    return df

def process_low_freq(df, col, min_freq=6):
    value_counts = df[col].value_counts()
    valid_values = value_counts[value_counts > min_freq].index
    df[col] = df[col].where(df[col].isin(valid_values))
    return df

def process_move_month(df):
    #비어있는경우 처리
    df.loc[df['전입월'] == 0, '전입월'] = pd.NA
    # 전입월 컬럼이 문자열인 경우 정수로 변환
    df['전입월'] = df['전입월'].astype('Int64')
    # 201107 이전의 값은 201107로 하단코딩
    df.loc[df['전입월'] < 201107, '전입월'] = 201107    
    df = process_low_freq(df, '전입월')
    df = df.rename(columns={'전입월': '이전주소'})
    return df

def sum_code(df):
    return df['체류자격'].replace({'D1': 'D1D5' , 'D5': 'D1D5'})
    # df=df.astype(str)
    # return df

def process_nationality(df_all):
    # NaN을 빈 문자열로 처리 후 문자열로 변환
    df_all['국적코드'] = df_all['국적코드'].fillna('').astype(str)

    # 월별 빈도 계산
    freq = df_all.groupby(['국적코드', '파일명']).size().reset_index(name='count')

    # 평균 빈도 계산
    avg_counts = freq.groupby('국적코드')['count'].mean()

    # 월평균 1000 이하 → ETC로 대체
    low_freq_codes = avg_counts[avg_counts <= 1000].index
    df_all['합쳐진 국적코드'] = df_all['국적코드'].where(~df_all['국적코드'].isin(low_freq_codes), 'ETC')

    return df_all

3. 각 컬럼별 가명처리 진행

In [192]:
df_all.columns
df_all.head()

,기준연월,연령,성별,내외국인구분,행정동코드,전입월,이전행정동코드,국적코드,체류자격,파일명
0,202503,3,2,1,77221,NaN,31377.0,241,A3,인구_202501.csv
1,202504,81,1,2,12308,NaN,NaN,941,A3,인구_202501.csv
2,202504,98,2,1,21465,202103.0,32453.0,104,A3,인구_202501.csv
3,202504,16,1,1,23509,202209.0,66938.0,808,C3,인구_202501.csv
4,202504,11,2,1,61470,202101.0,12964.0,437,D1,인구_202501.csv


In [193]:
# 연령 처리
df_age = process_age(df_all[['연령']].copy())
df_age_counts=df_age.value_counts().reset_index()
# 행정동코드 처리
df_addr = process_low_freq(df_all[['행정동코드']].copy(), '행정동코드').value_counts().reset_index()
# 전입월 → 이전주소 처리
df_prev_addr = process_move_month(df_all[['전입월']].copy())
df_prev_addr_counts = df_prev_addr.value_counts(dropna=False).reset_index()
df_prev_addr_counts.columns = ['이전주소', '빈도수']

# 이전행정동코드 처리
df_prev_dong_series = process_low_freq(df_all[['이전행정동코드']].copy(), '이전행정동코드')
df_prev_dong_counts = df_prev_dong_series.value_counts().reset_index()

# 국적코드 병합
df_dist = process_nationality(df_all.copy())
df_dist_counts = df_dist['합쳐진 국적코드'].value_counts().reset_index()
df_dist_counts.columns = ['합쳐진 국적코드', '빈도수']
# 체류자격 처리
df_cert_series = sum_code(df_all[['체류자격']].copy())
df_cert_counts = df_cert_series.value_counts().reset_index()
df_cert_counts.columns = ['체류자격', '빈도수']


In [216]:
df_dist_counts

,합쳐진 국적코드,빈도수
0,436,81084
1,713,81072
2,619,81000
3,648,80980
4,808,80900
5,788,80860
6,268,80808
7,195,80779
8,479,80764
9,969,80664


4. 월별 가명처리 데이터분포 확인

In [217]:
print('연령처리')
df_age_counts.to_csv(f"{output_dir}/가명처리_연령대.csv", index=False)
print('행정동')
df_addr.to_csv(f"{output_dir}/가명처리_행정동코드.csv", index=False)
print('전입월')
df_prev_addr_counts.to_csv(f"{output_dir}/가명처리_이전주소.csv", index=False)
print('이전행정동코드')
df_prev_dong_counts.to_csv(f"{output_dir}/가명처리_이전행정동코드.csv", index=False)
print('국적코드')
df_dist_counts.to_csv(f"{output_dir}/가명처리_합쳐진_국적코드.csv", index=False)
print('체류자격')
df_cert_counts.to_csv(f"{output_dir}/가명처리_체류자격.csv", index=False)

연령처리
행정동
전입월
이전행정동코드
국적코드
체류자격


In [ ]:

df_national_distribution = process_nationality(df_all[['합쳐진 국적코드']]).value_counts().reset_index()

5. 전체 데이터 가명처리 데이터분포 확인

In [ ]:
df_final = df_all.copy()
df_final = process_age(df_final)
df_final = process_low_freq(df_final, '행정동코드')
df_final = process_low_freq(df_final, '이전행정동코드')
df_final = process_move_month(df_final)
df_final = process_nationality(df_final)

drop_cols = ['국적코드', '파일명']
df_final = df_final.drop(columns=drop_cols)




,기준연월,성별,내외국인구분,행정동코드,이전주소,이전행정동코드,국적코드,체류자격,파일명,연령대,합쳐진 국적코드
0,202503,2,1,77221,<NA>,31377.0,241,A3,인구_202501.csv,1,241
1,202504,1,2,12308,<NA>,NaN,941,A3,인구_202501.csv,14,941
2,202504,2,1,21465,202103,32453.0,104,A3,인구_202501.csv,14,104
3,202504,1,1,23509,202209,66938.0,808,C3,인구_202501.csv,3,808
4,202504,2,1,61470,202101,12964.0,437,D1,인구_202501.csv,2,437


In [220]:
# # 원본 컬럼 제거
drop_cols = ['국적코드', '파일명']
df_final = df_final.drop(columns=drop_cols)
df_final.head()


,기준연월,성별,내외국인구분,행정동코드,이전주소,이전행정동코드,체류자격,연령대,합쳐진 국적코드
0,202503,2,1,77221,<NA>,31377.0,A3,1,241
1,202504,1,2,12308,<NA>,NaN,A3,14,941
2,202504,2,1,21465,202103,32453.0,A3,14,104
3,202504,1,1,23509,202209,66938.0,C3,3,808
4,202504,2,1,61470,202101,12964.0,D1,2,437
